# Initialization

Test notebook for the C-MAPPS benchmark. Test the Tunable Model class architecture

First we import the necessary packages and create the global variables.

In [28]:
import math
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import custom_scores
import CMAPSAuxFunctions
from tunableModel import TunableModel

from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras import backend as K
from keras import regularizers

from scipy.optimize import differential_evolution

# Create models

Create different types of models to be tested with the Tunable Model class

In [184]:
#Clear the previous tensorflow graph
K.clear_session()

FilterN = 10
FilterL = 10

def RULmodel_SN_1(input_shape):
    
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(30, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', name='fc1'))
    model.add(Dropout(0.4))
    model.add(Dense(10, activation='relu', kernel_initializer='glorot_normal', name='fc2'))
    model.add(Dropout(0.1))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model

def RULmodel_SN_2(input_shape):
    
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', name='fc1'))
    #model.add(Dense(30, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
    #                kernel_regularizer=regularizers.l2(0.1), name='fc1'))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model


def RULModel_CNN_1(input_shape):
    
    #Create a sequential model
    model = Sequential()
    
    print(input_shape)
    
    #Add the layers for the model
    model.add(Conv2D(1, (FilterN, FilterL), input_shape=input_shape, padding='same', kernel_initializer='glorot_normal', activation='relu', name='cl1'))
    model.add(Dropout(0.6))
    model.add(MaxPooling2D(pool_size=5, name='pl1'))
    model.add(Flatten())
    model.add(Dense(20, activation='relu', kernel_initializer='glorot_normal', name='fc1'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model


def RULCNNModel(TW, FeatureN):
    
    input_layer = Input(shape=(TW, FeatureN))
    y = Reshape((TW, FeatureN, 1), input_shape=(TW, FeatureN, ),name = 'Reshape')(input_layer)

    y = Conv2D(FilterN, FilterL, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='C1')(y)
    y = Conv2D(FilterN, FilterL, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='C2')(y)
    y = Conv2D(FilterN, FilterL, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='C3')(y)
    y = Conv2D(FilterN, FilterL, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='C4')(y)
    #y = Convolution2D(FilterN, FilterL, 1, border_mode='same', init='glorot_normal', activation='tanh', name='C5')(y)
    #y = Convolution2D(FilterN, FilterL, 1, border_mode='same', init='glorot_normal', activation='tanh', name='C6')(y)
    
    y = Conv2D(1, 3, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='Clast')(y)  
    
    y = Reshape((TW,14))(y)
    y = Flatten()(y)
    y = Dropout(0.5)(y)
    
    #y = Dense(100, activation='tanh', init='glorot_normal', activity_regularizer=keras.regularizers.l2(0.01),)(y)
    y = Dense(100,activation='tanh', kernel_initializer='glorot_normal', name='fc')(y)
    y = Dense(1)(y)
    
    model = Model(inputs = input_layer, outputs = y, name='RUL_CNN_Model')
    
    return model

# Load the data

In [198]:
#Shared parameters for the models
optimizer = Adam(lr=0, beta_1=0.5)
lossFunction = "mean_squared_error"
metrics = ["mse"]

#Selected as per CNN paper
features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']

selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])
#selected_indices = np.array([7, 8, 9, 12, 16, 17, 20])
#selected_indices = np.array(list(range(1,22)))

selected_features = list(features[i] for i in selected_indices-1)

print(selected_features)

windowSize = 30
stride = 1
constRUL = 125


dataFolder = '../CMAPSSData'

#Create and compile the models
nFeatures = len(selected_features)
shapeSN = nFeatures*windowSize
modelRULSN = RULmodel_SN_2(shapeSN)
modelRULSN.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

min_max_scaler = MinMaxScaler(feature_range=(-1, 1))

tModel = TunableModel('ModelRUL_SN_1', modelRULSN, selected_features, dataFolder, 'keras', window_size=windowSize,
                      scaler = min_max_scaler, window_stride=stride, constRul = constRUL)
tModel.changeDataset(1)
tModel.loadData(verbose=1, rectify_labels = False)
tModel.printData()

['T24', 'T30', 'T50', 'P30', 'Nf', 'Nc', 'Ps30', 'phi', 'NRf', 'NRc', 'BPR', 'htBleed', 'W31', 'W32']
Loading data for dataset 1 with window_size of 30, stride of 1 and constRUL of 125. Cros-Validation ratio 0
Data loaded for dataset 1
Printing shapes

Training data (X, y)
(17731, 420)
(17731, 1)
Testing data (X, y)
(100, 420)
(100, 1)
Printing first 5 elements

Training data (X, y)
[[-0.58075601 -0.0455243  -0.27982732 ... -0.81818182  0.43307087
   0.4679733 ]
 [-0.35395189  0.0629156  -0.18014129 ... -0.45454545  0.25984252
   0.25294702]
 [-0.21649485 -0.13299233 -0.13854003 ... -0.45454545  0.38582677
   0.72049425]
 [-0.21649485 -0.39897698 -0.2299843  ... -0.45454545  0.08661417
   0.29640676]
 [-0.20274914 -0.39590793 -0.05926217 ... -0.45454545  0.05511811
   0.17880983]]
[[125.]
 [125.]
 [125.]
 [125.]
 [125.]]
Testing data (X, y)
[[-0.65635739 -0.10946292 -0.48312402 ... -0.27272727  0.05511811
   0.30947309]
 [ 0.03780069 -0.07365729 -0.27629513 ... -0.63636364  0.05511811


In [199]:
#tModel.printData(printTop=False)
#tModel.windowSize = 20
#tModel.changeDataset(2)
#tModel.loadData(verbose=1, crossValRatio=0.1)
#tModel.printData(printTop=False)
tModel.getModelDescription()
#ruldcnn = RULCNNModel(tModel.windowSize, len(selected_features))
#ruldcnn.summary()
#tModel.trimmedRUL_train
#display(tModel.df_train)

Description for model: ModelRUL_SN_1
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
fc1 (Dense)                  (None, 20)                8420      
_________________________________________________________________
out (Dense)                  (None, 1)                 21        
Total params: 8,441
Trainable params: 8,441
Non-trainable params: 0
_________________________________________________________________


# Train the model and test some Tunable Model functionalities

In [200]:
lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)
#lrate = None

#print(tModel.currentModelName)
#tModel.getAllModelsDescription()
#tModel.getModelDescription('RULModel_CNN_1')
#tModel.getModelDescription(tModel.currentModelName)
#tModel.getModelDescription()
#tModel.setCurrentModel('RULmodel_SN_1')
#tModel.getModelDescription()
#Create and compile the models
nFeatures = len(selected_features)
shapeSN = nFeatures*windowSize
modelRULSN = RULmodel_SN_2(shapeSN)
modelRULSN.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)
tModel.changeModel('ModelRUL_SN_1', modelRULSN, 'keras')
tModel.epochs = 100
tModel.trainModel(learningRateScheduler=lrate, verbose=1)

Epoch 1/100
17731/17731 [==============================] - 0s 10us/step - loss: 7502.1268 - mean_squared_error: 7502.1268
Epoch 2/100
17731/17731 [==============================] - 0s 7us/step - loss: 5894.6351 - mean_squared_error: 5894.6351
Epoch 3/100
17731/17731 [==============================] - 0s 8us/step - loss: 4271.8114 - mean_squared_error: 4271.8114
Epoch 4/100
17731/17731 [==============================] - 0s 8us/step - loss: 2914.6898 - mean_squared_error: 2914.6898
Epoch 5/100
17731/17731 [==============================] - 0s 8us/step - loss: 2045.4102 - mean_squared_error: 2045.4102
Epoch 6/100
17731/17731 [==============================] - 0s 7us/step - loss: 1598.2312 - mean_squared_error: 1598.2312
Epoch 7/100
17731/17731 [==============================] - 0s 8us/step - loss: 1313.7235 - mean_squared_error: 1313.7235
Epoch 8/100
17731/17731 [==============================] - 0s 7us/step - loss: 1071.7858 - mean_squared_error: 1071.7858
Epoch 9/100
17731/17731 [======

17731/17731 [==============================] - 0s 7us/step - loss: 211.2034 - mean_squared_error: 211.2034
Epoch 70/100
17731/17731 [==============================] - 0s 7us/step - loss: 210.9755 - mean_squared_error: 210.9755
Epoch 71/100
17731/17731 [==============================] - 0s 7us/step - loss: 210.2298 - mean_squared_error: 210.2298
Epoch 72/100
17731/17731 [==============================] - 0s 8us/step - loss: 209.8783 - mean_squared_error: 209.8783
Epoch 73/100
17731/17731 [==============================] - 0s 7us/step - loss: 209.4607 - mean_squared_error: 209.4607
Epoch 74/100
17731/17731 [==============================] - 0s 9us/step - loss: 208.8725 - mean_squared_error: 208.8725
Epoch 75/100
17731/17731 [==============================] - 0s 10us/step - loss: 208.5156 - mean_squared_error: 208.5156
Epoch 76/100
17731/17731 [==============================] - 0s 8us/step - loss: 207.8179 - mean_squared_error: 207.8179
Epoch 77/100
17731/17731 [==========================

In [201]:
tModel.evaluateModel(["rhs"], round=2, setLimits=[7,145])
print("scores")

#print(tModel.y_pred)

cScores = tModel.scores
rmse = math.sqrt(cScores['score_1'])
rmse2 = cScores['rmse']
rhs = cScores['rhs']
time = tModel.trainTime

i = 1
for prediction in tModel.y_pred:
    print('Engine {}, RUL {}, Rounded RUL {}, Real RUL {}'.format(i, prediction, 
                                                                    tModel.y_pred_rectified[i-1], 
                                                                    tModel.y_test[i-1]))
    i = i+1

print(cScores)
print("RMSE: {}".format(rmse))
print("RMSE2: {}".format(rmse2))
print("RHS: {}".format(rhs))
print("Time : {} seconds".format(time))

100/100 [==============================] - 0s 339us/step
limits
scores
Engine 1, RUL [112.20074], Rounded RUL 112.0, Real RUL [112.]
Engine 2, RUL [118.57265], Rounded RUL 118.0, Real RUL [98.]
Engine 3, RUL [62.25758], Rounded RUL 62.0, Real RUL [69.]
Engine 4, RUL [86.58604], Rounded RUL 86.0, Real RUL [82.]
Engine 5, RUL [100.07608], Rounded RUL 100.0, Real RUL [91.]
Engine 6, RUL [104.89945], Rounded RUL 104.0, Real RUL [93.]
Engine 7, RUL [105.584496], Rounded RUL 105.0, Real RUL [91.]
Engine 8, RUL [89.10135], Rounded RUL 89.0, Real RUL [95.]
Engine 9, RUL [117.735176], Rounded RUL 117.0, Real RUL [111.]
Engine 10, RUL [91.12663], Rounded RUL 91.0, Real RUL [96.]
Engine 11, RUL [89.218216], Rounded RUL 89.0, Real RUL [97.]
Engine 12, RUL [100.360596], Rounded RUL 100.0, Real RUL [124.]
Engine 13, RUL [84.58367], Rounded RUL 84.0, Real RUL [95.]
Engine 14, RUL [107.44247], Rounded RUL 107.0, Real RUL [107.]
Engine 15, RUL [107.584404], Rounded RUL 107.0, Real RUL [83.]
Engine 16, 

In [ ]:
def nn_optmize_fun(x, selected_features=['T24', 'T30', 'T50', 'P30', 'Nf', 'Nc', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'htBleed', 'W31', 'W32'], datasetNumber = '1', scaler = None, verbose=0, epochs=250, 
                  saveToFile = None, iterations = 0):
    
    #Clear the previous tensorflow graph
    K.clear_session()
    
    #Extract the tunning variables from the input vector
    #Round the values to the nearest integer since this implementation is for real numbers
    x = x.astype(int)
    windows_size = x[0]
    window_stride = x[1]
    constantRUL = x[2]
    
    if iterations == 0:
        print("Creating model")
    #Shared parameters for the models
    optimizer = Adam(lr=0, beta_1=0.5)
    lossFunction = "mean_squared_error"
    metrics = ["mse"]
    
    #Define the model
    nFeatures = len(selected_features)
    shapeSN = nFeatures*windows_size
    modelRULSN = RULmodel_SN_1(shapeSN)
    modelRULSN.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

    #load the data using the selected parameters
    tModel = TunableModel(selected_features, scaler = min_max_scaler, window_stride = window_stride, 
                          window_size = windows_size, constRul = constantRUL, datasetNumber = datasetNumber, 
                         epochs=epochs)
    
    tModel.loadData()
    
    #Add the models to the tunableModel object
    tModel.addModel("RUL_SN_1_optmizable", modelRULSN)
    
    if iterations == 0:
        print("Training model")
    #Train the model
    lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)
    tModel.trainCurrentModel(learningRateScheduler=lrate, verbose=verbose)
    time = tModel.getModelTimes(tModel.currentModelName)
    
    if iterations == 0:
        print("Training time {}".format(time))
    
    if iterations == 0:
        print("Assesing model performance")
    #Assess the model performance
    tModel.evaluateCurrentModel(["rhs"], crossValidation=True)
    cScores = tModel.getModelScores(tModel.currentModelName)
    rmse = math.sqrt(cScores['score_1'])
    rhs = cScores['rhs']
    #print("The score for this model is: {}".format(rmse))
    
    msgStr = "The model variables are " + str(x) + "\tThe scores are: [RMSE:{:.4f}, RHS:{:.4f}]\n".format(rmse, rhs)
    
    if saveToFile is not None:
        #print(msgStr)
        saveToFile.write(msgStr)
    else:
        print(msgStr)
    
    #Return RMSE as the performance metric to steer the search
    return rmse

In [ ]:
def plotRUL(cycles, rulArray, engineUnit):
    
    plt.clf()
    plt.plot(cycles, rulArray, 'bo-', label='RUL')
    plt.xlabel("Time (Cycle)")
    plt.ylabel("RUL")
    plt.title("Test Engine Unit #{}".format(engineUnit))
    
    plt.show()
    
def plotRUL_Engine(tModel, engineUnit, selectedFeatures, time_window, constRUL, stride, y_test):

    df = tModel.df_test
    df_engine = df[df['Unit Number'] == engineUnit]
    
    X_test2, _, _, _ = CMAPSAuxFunctions.create_windowed_data(df_engine, selected_features, 'test', time_window=time_window, 
                                                     constRUL=constRUL, stride=stride)
    
    X_test2 = tModel.dataScaler.transform(X_test2)
    nnPred = tModel.model.predict(X_test2)
    
    maxCycle = X_test2.shape[0]
    faultCycle = tModel.y_test[engineUnit-1]
    cycles = np.arange(maxCycle)
    rulArray = np.arange(faultCycle, maxCycle+faultCycle)
    rulArray[rulArray > constRUL] = constRUL
    rulArray = np.flipud(rulArray)
    
    plotRUL(cycles, rulArray, engineUnit)
    plt.plot(cycles, nnPred, 'go-', label='NN Pred')
    plt.legend()

In [ ]:
plotRUL_Engine(tModel, engineUnit, selectedFeatures, time_window, stride, constRUL)